# 불량 이미지 검출 프로그램

## 기본 세팅
* 기준 이미지의 파일명 : original.jpg
* 기타 이미지의 파일명은 무작위로 설정

## 기능 명세
* 기준 이미지와 기타 모든 이미지를 비교
* 기준 이미지와 다른 이미지가 발견되면 해당 이미지의 파일명 출력
* 디렉토리 내에 불량 이미지가 총 몇 개인지 출력
* 불량 이미지들의 복사본을 Defects 폴더에 저장

In [1]:
import os
import cv2
import numpy as np

## 디렉토리의 모든 이미지 로드하기

In [2]:
# 불러올 이미지들을 저장할 리스트
img = []

# 이미지들의 파일명들을 저장할 리스트
imageNames = []

# 이미지 불러오기
for i, imageName in enumerate(os.listdir('TestData/')):
    # 불러올 이미지의 경로를 변수에 저장
    path = 'TestData/' + imageName
    
    # 이미지 불러오기
    img.append(cv2.imread(path, cv2.IMREAD_COLOR))
    
    # 불러온 이미지의 파일명을 리스트에 저장
    imageNames.append(imageName)
    
    # 기준 이미지를 리스트의 선두로 가져오기
    if imageName == 'original.jpg':
        temp = img[0]; img[0] = img[i]; img[i] = temp
        temp = imageNames[0]; imageNames[0] = imageNames[i]; imageNames[i] = temp
    
for i in imageNames:
    print(i)

original.jpg
.DS_Store
jljsalkdfja.jpg
kblaskd.jpg
kljsdklj30.jpg
9349024ksndk.jpg
kfdskljsdfl30402.jpg
92390sdkljf.jpg
skljalkad.jpg
ljkdjklsajlk.jpg
newlkljeoij.jpg
kljdsjkl2000101.jpg
kldskl230.jpg
kljwekdfsklj.jpg
29034skjf.jpg
jalsjla200030.jpg


## 이미지 비교 함수 정의

In [3]:
def compareImage(original, compare):
    imgs = [original, compare]
    hists = []
    hists.clear()
    
    for i in imgs:
        hsv = cv2.cvtColor(i, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
        cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
        hists.append(hist)

    # 카이 제곱 알고리즘으로 이미지 비교 수행
    ret = cv2.compareHist(hists[0], hists[1], cv2.HISTCMP_CHISQR)

    # 비교 결과 리턴
    return ret

## 기준 이미지와 다른 이미지 찾아내기(일반 디렉토리에서 실행하는 경우)

In [ ]:
defectImgsName = []

for (i, targetImage) in enumerate(img):
    # 비교 함수 호출
    result = compareImage(img[0], targetImage)
    # 비교 결과 수치 출력(실행 속도 향상을 위해 주석 처리)
    # print(imageNames[i] + '\t\t' + str(result))
    # 불량 판정(아래 if문의 조건식을 수정하여 불량 판정 기준 변경)
    if result > 0.02:
        defectImgsName.append(imageNames[i])

if len(defectImgsName) == 0:
    print("모든 이미지가 정상으로 판별되었습니다.")
else:
    for i, defectImage in enumerate(imageNames):
        if defectImage in defectImgsName:
            print(imageNames[i], end = '\t'); print("이미지는 불량으로 판별되었습니다.")
    print("\n불량 이미지는 총 %d개로 확인되었습니다." % len(defectImgsName))

## 기준 이미지와 다른 이미지 찾아내기(Github 디렉토리에서 실행하는 경우)

In [4]:
defectImgsName = []

for (i, targetImage) in enumerate(img):
    if imageNames[i] != '.DS_Store':
        # 비교 함수 호출
        result = compareImage(img[0], targetImage)
        # 비교 결과 수치 출력(실행 속도 향상을 위해 주석 처리)
        # print(imageNames[i] + '\t\t' + str(result))
        # 불량 판정(아래 if문의 조건식을 수정하여 불량 판정 기준 변경)
        if result > 0.02:
            defectImgsName.append(imageNames[i])

if len(defectImgsName) == 0:
    print("모든 이미지가 정상으로 판별되었습니다.")
else:
    for i, defectImage in enumerate(imageNames):
        if defectImage in defectImgsName:
            print(imageNames[i], end = '\t'); print("이미지는 불량으로 판별되었습니다.")
    print("\n불량 이미지는 총 %d개로 확인되었습니다." % len(defectImgsName))

skljalkad.jpg	이미지는 불량으로 판별되었습니다.
ljkdjklsajlk.jpg	이미지는 불량으로 판별되었습니다.
jalsjla200030.jpg	이미지는 불량으로 판별되었습니다.

불량 이미지는 총 3개로 확인되었습니다.


## 불량 이미지들의 복사본을 새 디렉토리에 저장

In [5]:
CheckDir = 0

if len(defectImgsName) != 0:
    for dirName in os.listdir('./'):
        if dirName == 'Defects':
            CheckDir = CheckDir + 1
            break
    if CheckDir == 0:
        os.mkdir('./Defects')

for i, searchImg in enumerate(imageNames):
    if searchImg in defectImgsName:
        filename = 'Defects/' + searchImg
        cv2.imwrite(filename, img[i])